1. Load the dataset provided. **(3 pts)**
   - Ensure all columns have the correct data type, e.g., `float` or `int` for numeric quantities, `object` for categorical variables, `datetime` for dates, etc.
   - Remove columns where all values are missing or equal to zero.
   - Describe the basic statistics (mean, minimum, maximum, standard deviation, etc.) of the numerical variables.

2. Perform all data pre-processing steps needed for the model (i.e., integration, cleaning, reduction, and transformation). **(7 pts)**  
   For each pre-processing step that you apply, dedicate at least one markdown cell next to the code to explain:
   - the step you are applying
   - why you need to do it
   - what is the result  

   **Note:** In some scenarios, it is acceptable to repeat steps; however, make sure your code is logical and efficient.

3. Organize your time-dependent data. In a separate markdown cell, summarize how you organize your time-dependent data. Motivate your strategy. You can include a figure if needed. **(4 pts)**
   - **Hint:** you can include a figure in markdown syntax as follows:  

4. Prepare your data for model selection and evaluation. **(4 pts)**  
   What is the experimental setup that you use to build the model? Please provide a figure that summarizes your setup where you clearly mark:
   - how do you split the available data for training, validation, or testing
   - what type of validation strategy do you use (e.g., hold out, cross-validation)

5. Choose a recurrent neural network architecture to build the prediction model (e.g., LSTM, GRU, etc.) and implement your solution. Motivate which hyperparameters you select or optimize. **(5 pts)**

6. Create plots of training and validation losses. Describe the trends you notice and whether they indicate underfitting or overfitting behaviors. **(2 pts)**

7. Evaluate the performance of the model. Describe which measures you use and discuss the results of the evaluation. **(3 pts)**

8. Create a plot contrasting the predictions of the model and the ground truth values from the test set. **(2 pts)**

In [1]:
import pandas as pd
import numpy as np

# Load datasets
train = pd.read_csv('energy_generation_train.csv')
test = pd.read_csv('energy_generation_test.csv')

def process(df, name):
    print(f"\n{'='*50}\nProcessing {name}\n{'='*50}")
    
    # Convert DateTime column (if present) to datetime
    if 'DateTime' in df.columns:
        df['DateTime'] = pd.to_datetime(df['DateTime'])
    
    # Identify numeric columns (everything except DateTime)
    numeric_cols = [c for c in df.columns if c != 'DateTime']
    
    # Remove columns where all values are either missing OR zero
    # (i.e., after filling NaN with 0, all values become 0)
    cols_to_drop = []
    for col in numeric_cols:
        if df[col].fillna(0).eq(0).all():
            cols_to_drop.append(col)
    
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
        print(f"Dropped columns (all missing/zero): {cols_to_drop}")
    
    # Remaining numeric columns
    remaining_numeric = [c for c in df.columns if c != 'DateTime' and c in numeric_cols]
    
    # Basic statistics for numerical variables
    if remaining_numeric:
        stats = df[remaining_numeric].describe().T[['mean','min','max','std']]
        stats['missing'] = df[remaining_numeric].isnull().sum()
        print("\nNumerical statistics:")
        print(stats.round(2))
    else:
        print("No numerical columns remaining.")
    
    print(f"\nFinal columns: {list(df.columns)}")
    return df

# Process both sets
train_clean = process(train, 'TRAIN Dataset')
test_clean = process(test, 'TEST Dataset')



Processing TRAIN Dataset
Dropped columns (all missing/zero): ['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 'Hydro Pumped Storage.1', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Other renewable']

Numerical statistics:
                     mean  min      max      std  missing
Fossil Gas        3512.53  0.0   9666.0  2256.92       27
Fossil Hard coal  1362.93  0.0   3912.0   999.45       27
Nuclear            433.14  0.0    490.0   135.73       26
Other              351.87  0.0  11638.0   292.84       26
Solar               33.59  0.0    252.0    56.98       26
Waste               62.45  0.0     89.0    17.87       27
Wind Offshore     1296.40  0.0   3339.0   966.51       26
Wind Onshore       431.28  0.0   1226.0   347.33       26

Final columns: ['DateTime', 'Fossil Gas', 'Fossil Hard coal', 'Nuclear', 'Other', 'Solar', 'Waste', 'Wind Offshore', 'Wind

In [18]:
with pd.option_context('display.max_columns', None, 'display.max_rows', None):
    display(train_df.head(100))
    display(train_df.tail(100))
    display(test_df.head(100))
    display(test_df.tail(100))


,DateTime,Biomass,Fossil Brown coal/Lignite,Fossil Coal-derived gas,Fossil Gas,Fossil Hard coal,Fossil Oil,Fossil Oil shale,Fossil Peat,Geothermal,Hydro Pumped Storage,Hydro Pumped Storage.1,Hydro Run-of-river and poundage,Hydro Water Reservoir,Marine,Nuclear,Other,Other renewable,Solar,Waste,Wind Offshore,Wind Onshore
0,2023-01-01 00:00:00,0.0,NaN,NaN,919.0,1225.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,161.0,NaN,1.0,69.0,1893.0,807.0
1,2023-01-01 00:15:00,0.0,NaN,NaN,921.0,1235.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,139.0,NaN,1.0,70.0,1367.0,741.0
2,2023-01-01 00:30:00,0.0,NaN,NaN,922.0,1249.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,139.0,NaN,1.0,69.0,1052.0,686.0
3,2023-01-01 00:45:00,0.0,NaN,NaN,966.0,1254.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,140.0,NaN,1.0,68.0,1090.0,682.0
4,2023-01-01 01:00:00,0.0,NaN,NaN,999.0,1258.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,137.0,NaN,1.0,68.0,1082.0,760.0
5,2023-01-01 01:15:00,0.0,NaN,NaN,993.0,1239.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,157.0,NaN,1.0,69.0,1088.0,769.0
6,2023-01-01 01:30:00,0.0,NaN,NaN,994.0,1235.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,484.0,161.0,NaN,1.0,70.0,1075.0,757.0
7,2023-01-01 01:45:00,0.0,NaN,NaN,999.0,1226.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,139.0,NaN,1.0,69.0,1060.0,727.0
8,2023-01-01 02:00:00,0.0,NaN,NaN,991.0,1228.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,139.0,NaN,1.0,69.0,1055.0,760.0
9,2023-01-01 02:15:00,0.0,NaN,NaN,968.0,1244.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,484.0,159.0,NaN,1.0,67.0,1067.0,766.0


,DateTime,Biomass,Fossil Brown coal/Lignite,Fossil Coal-derived gas,Fossil Gas,Fossil Hard coal,Fossil Oil,Fossil Oil shale,Fossil Peat,Geothermal,Hydro Pumped Storage,Hydro Pumped Storage.1,Hydro Run-of-river and poundage,Hydro Water Reservoir,Marine,Nuclear,Other,Other renewable,Solar,Waste,Wind Offshore,Wind Onshore
36092,2024-01-30 23:00:00,0.0,NaN,NaN,5019.0,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,187.0,NaN,0.0,74.0,754.0,383.0
36093,2024-01-30 23:15:00,0.0,NaN,NaN,4622.0,1853.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,189.0,NaN,0.0,73.0,747.0,372.0
36094,2024-01-30 23:30:00,0.0,NaN,NaN,4541.0,1786.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,186.0,NaN,0.0,72.0,679.0,262.0
36095,2024-01-30 23:45:00,0.0,NaN,NaN,4599.0,1680.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,187.0,NaN,0.0,74.0,658.0,214.0
36096,2024-01-31 00:00:00,0.0,NaN,NaN,4545.0,1664.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,186.0,NaN,0.0,74.0,646.0,222.0
36097,2024-01-31 00:15:00,0.0,NaN,NaN,4559.0,1738.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,487.0,186.0,NaN,0.0,73.0,618.0,191.0
36098,2024-01-31 00:30:00,0.0,NaN,NaN,4528.0,1734.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,185.0,NaN,0.0,73.0,690.0,175.0
36099,2024-01-31 00:45:00,0.0,NaN,NaN,4502.0,1696.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,186.0,NaN,0.0,72.0,740.0,171.0
36100,2024-01-31 01:00:00,0.0,NaN,NaN,4410.0,1689.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,186.0,NaN,0.0,72.0,768.0,241.0
36101,2024-01-31 01:15:00,0.0,NaN,NaN,4217.0,1611.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,186.0,NaN,0.0,71.0,767.0,267.0


,DateTime,Biomass,Fossil Brown coal/Lignite,Fossil Coal-derived gas,Fossil Gas,Fossil Hard coal,Fossil Oil,Fossil Oil shale,Fossil Peat,Geothermal,Hydro Pumped Storage,Hydro Pumped Storage.1,Hydro Run-of-river and poundage,Hydro Water Reservoir,Marine,Nuclear,Other,Other renewable,Solar,Waste,Wind Offshore,Wind Onshore
0,2024-02-01 00:00:00,0.0,NaN,NaN,1795.0,950.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,185.0,NaN,0.0,69.0,3260.0,1207.0
1,2024-02-01 00:15:00,0.0,NaN,NaN,1586.0,958.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,206.0,NaN,0.0,70.0,3269.0,1199.0
2,2024-02-01 00:30:00,0.0,NaN,NaN,1349.0,1055.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,208.0,NaN,0.0,69.0,3272.0,1180.0
3,2024-02-01 00:45:00,0.0,NaN,NaN,1264.0,997.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,184.0,NaN,0.0,64.0,3275.0,1180.0
4,2024-02-01 01:00:00,0.0,NaN,NaN,1278.0,982.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,184.0,NaN,0.0,61.0,3277.0,1183.0
5,2024-02-01 01:15:00,0.0,NaN,NaN,1241.0,985.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,185.0,NaN,0.0,67.0,3269.0,1177.0
6,2024-02-01 01:30:00,0.0,NaN,NaN,1227.0,991.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,186.0,NaN,0.0,73.0,3267.0,1172.0
7,2024-02-01 01:45:00,0.0,NaN,NaN,1216.0,989.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,186.0,NaN,0.0,73.0,3259.0,1165.0
8,2024-02-01 02:00:00,0.0,NaN,NaN,1213.0,979.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,187.0,NaN,0.0,72.0,3223.0,1173.0
9,2024-02-01 02:15:00,0.0,NaN,NaN,1215.0,938.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,186.0,NaN,0.0,72.0,3176.0,1128.0


,DateTime,Biomass,Fossil Brown coal/Lignite,Fossil Coal-derived gas,Fossil Gas,Fossil Hard coal,Fossil Oil,Fossil Oil shale,Fossil Peat,Geothermal,Hydro Pumped Storage,Hydro Pumped Storage.1,Hydro Run-of-river and poundage,Hydro Water Reservoir,Marine,Nuclear,Other,Other renewable,Solar,Waste,Wind Offshore,Wind Onshore
33216,2025-01-30 23:00:00,0.0,NaN,NaN,5969.0,3813.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,197.0,NaN,0.0,75.0,1375.0,341.0
33217,2025-01-30 23:15:00,0.0,NaN,NaN,5464.0,3762.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,195.0,NaN,0.0,74.0,1353.0,308.0
33218,2025-01-30 23:30:00,0.0,NaN,NaN,5207.0,3753.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,485.0,197.0,NaN,0.0,75.0,1288.0,363.0
33219,2025-01-30 23:45:00,0.0,NaN,NaN,5172.0,3761.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,196.0,NaN,0.0,75.0,1285.0,238.0
33220,2025-01-31 00:00:00,0.0,NaN,NaN,4906.0,3754.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,190.0,NaN,0.0,74.0,1334.0,257.0
33221,2025-01-31 00:15:00,0.0,NaN,NaN,4912.0,3804.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,487.0,186.0,NaN,0.0,74.0,1278.0,270.0
33222,2025-01-31 00:30:00,0.0,NaN,NaN,4905.0,3800.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,185.0,NaN,0.0,74.0,1240.0,248.0
33223,2025-01-31 00:45:00,0.0,NaN,NaN,4673.0,3798.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,195.0,NaN,0.0,75.0,1325.0,236.0
33224,2025-01-31 01:00:00,0.0,NaN,NaN,4709.0,3797.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,192.0,NaN,0.0,75.0,1304.0,292.0
33225,2025-01-31 01:15:00,0.0,NaN,NaN,4885.0,3799.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,486.0,189.0,NaN,0.0,75.0,1247.0,306.0
